In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
'''
UTILIDADES
'''

from sklearn import datasets
import matplotlib.pyplot as plt

def generate_data(data_type, noise=0.2):
    """
    Generate a binary dataset with distribution data_type

    Arguments:
    data_type -- distribution of dataset {moons,circles,blobs}

    Returns:
    X -- features
    Y -- labels
    """ 
    np.random.seed(0)
    if data_type == 'moons':
        X, Y = datasets.make_moons(200, noise=noise)
    elif data_type == 'circles':
        X, Y = sklearn.datasets.make_circles(200, noise=noise)
    elif data_type == 'blobs':
        X, Y = sklearn.datasets.make_blobs(centers=2, cluster_std=noise)
    return X, Y

def linear_activation(W, b, X):
    z = np.dot(W,X) + b
    
    return z

def sigmoid(z):
    return 1. / (1. + np.exp(-z))

def predict_multilayer(parameters,X):
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    Z1 = linear_activation(W1,b1,X)
    A1 = sigmoid(Z1)
    
    Z2 = linear_activation(W2,b2,A1)
    A2 = sigmoid(Z2)

    return np.round(A2)

def visualize_lr(parameters, X, Y):
    X = X.T
    
    # Set min and max values and give it some padding
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole gid
    #Z = pred_func(W,b,np.c_[xx.ravel(), yy.ravel()])
    Z = predict_multilayer(parameters, np.c_[xx.ravel(), yy.ravel()].T)
    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.figure(figsize=(7,5))
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    
    color= ['blue' if y == 1 else 'red' for y in np.squeeze(Y)]
    plt.scatter(X[:,0], X[:,1], color=color)
    
    plt.show()

<hr>
## 1. Cargue del dataset

In [ ]:
'''
Dataset de ejemplo para clasificacion binaria
'''
import sklearn
from sklearn.datasets import make_blobs, make_moons

dataset = generate_data('circles')
x = dataset[0]
y = dataset[1]

nx,m = x.T.shape

colors = ['red' if label == 1 else 'blue' for label in y]

In [ ]:
plt.figure(figsize=(7,5))
plt.scatter(x[:,0], x[:,1], color=colors)

plt.show()

#x = x.T

<hr>
## 2. MLP: Implementacion explicita

In [ ]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [ ]:
def d_sigmoid(z):
    return tf.nn.sigmoid(z)*(1.-tf.nn.sigmoid(z))

def loss(y, a):
    return -(y * tf.log(a) + (1.-y) * tf.log(1.-a))

In [ ]:
reset_graph()

learning_rate = 0.6

#dataset en forma de tensores
X = tf.constant(x.T, dtype=tf.float32, name="X")
Y = tf.constant(y, dtype=tf.float32, name="Y")

nx = x.shape[1] #numero de caracteristicas de la capa de 
n_hidden_layer = 10

'''
parametros de la red
'''

W1 = tf.Variable(tf.random_uniform([n_hidden_layer, nx]), name="W1", dtype=tf.float32)
b1 = tf.Variable(tf.random_uniform([n_hidden_layer,1]), name="b1", dtype=tf.float32)

W2 = tf.Variable(tf.random_uniform([1, n_hidden_layer]), name="W2", dtype=tf.float32)
b2 = tf.Variable(tf.random_uniform([1, 1]), name="b2", dtype=tf.float32)

'''
grafo de computo
'''

'''
forward propagation
'''
Z1 = tf.matmul(W1, X) + b1
A1 = tf.nn.sigmoid(Z1)

Z2 = tf.matmul(W2, A1) + b2
A2 = tf.nn.sigmoid(Z2)

'''
back propagation
'''
dZ2 = A2 - Y
dW2 = tf.matmul(dZ2, tf.transpose(A1)) / m 
db2 = tf.reduce_sum(dZ2, axis=1, keep_dims=True) / m

dZ1 = tf.multiply(tf.matmul(tf.transpose(W2), dZ2), d_sigmoid(Z1))
dW1 = tf.matmul(dZ1, tf.transpose(X)) / m
db1 = tf.reduce_sum(dZ1, axis=1, keep_dims=True) / m

'''
update
'''
training_W1 = tf.assign(W1, W1 - learning_rate*dW1)
training_b1 = tf.assign(b1, b1 - learning_rate*db1)
training_W2 = tf.assign(W2, W2 - learning_rate*dW2)
training_b2 = tf.assign(b2, b2 - learning_rate*db2)
'''
cost
'''
J = tf.reduce_mean(loss(y,A2))

<hr>
## 3. Entrenamiento del modelo

In [ ]:
init = tf.global_variables_initializer()

n_epochs = 10000

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if(epoch % 1000 == 0):
            print ("Epoca ", epoch, "Error: ", J.eval(), b1.eval().shape)
        
        sess.run([training_W1, training_b1, training_W2, training_b2])
        
        best_W1 = W1.eval()
        best_b1 = b1.eval()
        best_W2 = W2.eval()
        best_b2 = b2.eval()

    print best_W1, best_b1
    print best_W2, best_b2
        

In [ ]:
parameters = {'W1':best_W1, 'b1':best_b1, 'W2':best_W2, 'b2':best_b2}
visualize_lr(parameters, x.T, y)

<hr>
## 4. Implementacion utilizando utilidades de TensorFlow

In [ ]:
reset_graph()

learning_rate = 0.6

#dataset en forma de tensores
X = tf.constant(x.T, dtype=tf.float32, name="X")
Y = tf.constant(y, dtype=tf.float32, name="Y")

nx = x.shape[1] #numero de caracteristicas de la capa de 
n_hidden_layer = 10

'''
parametros de la red
'''

W1 = tf.Variable(tf.random_uniform([n_hidden_layer, nx]), name="W1", dtype=tf.float32)
b1 = tf.Variable(tf.random_uniform([n_hidden_layer,1]), name="b1", dtype=tf.float32)

W2 = tf.Variable(tf.random_uniform([1, n_hidden_layer]), name="W2", dtype=tf.float32)
b2 = tf.Variable(tf.random_uniform([1, 1]), name="b", dtype=tf.float32)

'''
grafo de computo
'''

'''
forward propagation
'''
Z1 = tf.matmul(W1, X) + b1
A1 = tf.nn.sigmoid(Z1)

Z2 = tf.matmul(W2, A1) + b2
A2 = tf.nn.sigmoid(Z2)

'''
error
'''
loss = -(y * tf.log(A2) + (1.-y) * tf.log(1.-A2))
J = tf.reduce_mean(loss)

'''
optimizacion
'''
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(J)

In [ ]:
init = tf.global_variables_initializer()

n_epochs = 10000

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if(epoch % 1000 == 0):
            print ("Epoca ", epoch, "Error: ", J.eval())
        
        sess.run(optimizer)
        
        best_W1 = W1.eval()
        best_b1 = b1.eval()
        best_W2 = W2.eval()
        best_b2 = b2.eval()

    print best_W1, best_b1
    print best_W2, best_b2

In [ ]:
parameters = {'W1':best_W1, 'b1':best_b1, 'W2':best_W2, 'b2':best_b2}
visualize_lr(parameters, x.T, y)

<hr>
## Trabajemos:
Agregue una nueva capa al perceptron multicapa.